<a href="https://colab.research.google.com/github/macroact-lab/robotics-book/blob/main/3_2_maicat_trot_gate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title 1. 라이브러리 및 로봇 모델 정의
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from mpl_toolkits.mplot3d import Axes3D
from collections import OrderedDict
from IPython.display import HTML

# --- 수학 함수 ---
def RpToTrans(R, p):
    return np.r_[np.c_[R, p], [[0, 0, 0, 1]]]

def TransToRp(T):
    T = np.array(T)
    return T[0:3, 0:3], T[0:3, 3]

def RPY(roll, pitch, yaw):
    Rx = np.array([[1, 0, 0], [0, np.cos(roll), -np.sin(roll)], [0, np.sin(roll), np.cos(roll)]])
    Ry = np.array([[np.cos(pitch), 0, np.sin(pitch)], [0, 1, 0], [-np.sin(pitch), 0, np.cos(pitch)]])
    Rz = np.array([[np.cos(yaw), -np.sin(yaw), 0], [np.sin(yaw), np.cos(yaw), 0], [0, 0, 1]])
    return np.matmul(np.matmul(Rx, Ry), Rz)

def solve_2link_intersection(p_start, p_end, r1, r2, bend_direction):
    vec = p_end - p_start
    d = np.linalg.norm(vec)
    max_reach = r1 + r2
    if d >= max_reach * 0.999: d = max_reach * 0.999
    min_reach = abs(r1 - r2)
    if d <= min_reach * 1.001: d = min_reach * 1.001

    a = (r1**2 - r2**2 + d**2) / (2*d)
    h = np.sqrt(max(0, r1**2 - a**2))

    x2 = p_start + a * (vec / d)
    perp_vec = np.array([-vec[2], 0, vec[0]])
    if np.linalg.norm(perp_vec) == 0: perp_vec = np.array([0, 0, 1])
    perp_vec = perp_vec / np.linalg.norm(perp_vec)

    if bend_direction == 'backward': perp_vec = -perp_vec
    return x2 + h * perp_vec

# --- 모델 클래스 ---
class MaicatModel:
    def __init__(self):
        self._len_pan = 0.016
        self._len_shoulder_f = 0.046
        self._len_leg_f = 0.063
        self._len_shoulder_r = 0.046
        self._len_bridge_r = 0.05
        self._len_leg_r = 0.025

        self._hip_x = 0.11
        self._hip_y = 0.084
        self._foot_x = 0.13
        self._foot_y = 0.11

        self._body_to_hip = OrderedDict()
        self._body_to_hip["FL"] = np.array([self._hip_x / 2.0, self._hip_y / 2.0, 0])
        self._body_to_hip["FR"] = np.array([self._hip_x / 2.0, -self._hip_y / 2.0, 0])
        self._body_to_hip["RL"] = np.array([-self._hip_x / 2.0, self._hip_y / 2.0, 0])
        self._body_to_hip["RR"] = np.array([-self._hip_x / 2.0, -self._hip_y / 2.0, 0])

        self._world_foot = OrderedDict()
        self._world_foot["FL"] = np.array([self._foot_x / 2.0, self._foot_y / 2.0, 0.0])
        self._world_foot["FR"] = np.array([self._foot_x / 2.0, -self._foot_y / 2.0, 0.0])
        self._world_foot["RL"] = np.array([-self._foot_x / 2.0, self._foot_y / 2.0, 0.0])
        self._world_foot["RR"] = np.array([-self._foot_x / 2.0, -self._foot_y / 2.0, 0.0])

In [ ]:
# @title 2. 동적 발 위치 계산 함수

def get_leg_points_dynamic(model, key, T_wb, current_foot_pos):
    """
    T_wb: 몸통의 변환 행렬
    current_foot_pos: 현재 발의 목표 위치 (X, Y, Z) - 움직일 수 있음!
    """
    points = []

    # 1. 힙 위치
    p_bh = model._body_to_hip[key]
    T_bh = RpToTrans(np.eye(3), p_bh)
    T_wh = np.dot(T_wb, T_bh)
    _, p_hip = TransToRp(T_wh)

    # 2. 발 위치 (입력받은 동적 위치 사용)
    p_foot = current_foot_pos

    # 3. Pan End
    side_sign = 1 if 'L' in key else -1
    R_wb, _ = TransToRp(T_wb)
    y_axis_body = R_wb[:, 1]
    p_pan = p_hip + y_axis_body * (model._len_pan * side_sign)

    points.append(p_hip)
    points.append(p_pan)

    # 4. Joint Solving (이전과 동일)
    if 'F' in key:
        p_knee = solve_2link_intersection(p_pan, p_foot, model._len_shoulder_f, model._len_leg_f, 'backward')
        points.append(p_knee)
        points.append(p_foot)
    else:
        l_long = model._len_shoulder_r + model._len_leg_r
        l_bridge = model._len_bridge_r
        p_temp_joint = solve_2link_intersection(p_pan, p_foot, l_long, l_bridge, 'forward')
        vec_thigh = p_temp_joint - p_pan
        u_t = vec_thigh / np.linalg.norm(vec_thigh)
        p_knee = p_pan + u_t * model._len_shoulder_r
        vec_bridge = p_foot - p_temp_joint
        p_ankle = p_knee + vec_bridge

        points.append(p_knee)
        points.append(p_ankle)
        points.append(p_foot)

    return np.array(points)

In [ ]:
# @title 3. 제자리 걸음 (Trot) 시뮬레이션
def create_marching_animation(frames=60):
    model = MaicatModel()

    fig = plt.figure(figsize=(10, 6))
    ax = fig.add_subplot(111, projection='3d')

    # 바닥 그리드
    xx, yy = np.meshgrid(np.linspace(-0.3, 0.3, 10), np.linspace(-0.2, 0.2, 10))
    ax.plot_wireframe(xx, yy, np.zeros_like(xx), color='gray', alpha=0.1)

    ax.set_xlim(-0.2, 0.2)
    ax.set_ylim(-0.2, 0.2)
    ax.set_zlim(0, 0.3)
    ax.set_xlabel('X (Front)')
    ax.set_ylabel('Y (Left)')
    ax.set_zlabel('Z (Up)')
    ax.view_init(elev=20, azim=-70)
    try: ax.set_box_aspect([1, 1, 1])
    except: pass

    # 그래픽 객체 초기화
    body_line, = ax.plot([], [], [], 'k-', linewidth=3, label='Body')
    head_point, = ax.plot([], [], [], 'go', markersize=10, label='Head')

    leg_lines = {}
    keys = ["FL", "FR", "RR", "RL"]
    for key in keys:
        line, = ax.plot([], [], [], color='blue', linewidth=4, marker='o', markersize=5, markeredgecolor='k')
        leg_lines[key] = line

    title = ax.set_title("Maicat Marching In Place (Trot)")

    def update(frame):
        # 속도 조절
        t = frame * 0.2

        # --- [Gait Logic] ---
        # 트로트(Trot): 대각선 다리끼리 같은 위상
        # Group A: FL, RR (위상 0)
        # Group B: FR, RL (위상 PI)

        step_height = 0.04 # 4cm 발 들기

        # 사인파의 양수 부분만 사용하여 발을 듬 (음수일 땐 0으로 바닥 지지)
        lift_A = step_height * max(0, np.sin(t))
        lift_B = step_height * max(0, np.sin(t + np.pi))

        # 각 발의 현재 목표 위치 계산
        current_feet = {}

        # FL & RR (Group A)
        current_feet["FL"] = model._world_foot["FL"].copy()
        current_feet["FL"][2] = lift_A

        current_feet["RR"] = model._world_foot["RR"].copy()
        current_feet["RR"][2] = lift_A

        # FR & RL (Group B)
        current_feet["FR"] = model._world_foot["FR"].copy()
        current_feet["FR"][2] = lift_B

        current_feet["RL"] = model._world_foot["RL"].copy()
        current_feet["RL"][2] = lift_B

        # --- [Body Motion] ---
        # 로봇이 발을 구를 때 몸통도 살짝 반동을 줌 (자연스러움 추가)
        z_bounce = 0.085 + 0.005 * np.cos(2 * t) # 2배 빠른 주기로 바운스

        orn = [0, 0, 0] # 회전 없음
        pos = [0, 0, z_bounce]

        R_wb = RPY(orn[0], orn[1], orn[2])
        T_wb = RpToTrans(R_wb, pos)

        # --- [Visualization Update] ---
        # 몸통
        hip_world_pos = []
        for key in keys:
            p_bh = model._body_to_hip[key]
            p_wh_homo = np.dot(T_wb, np.append(p_bh, 1))
            hip_world_pos.append(p_wh_homo[:3])
        hip_pos_arr = np.array(hip_world_pos)

        bx = np.append(hip_pos_arr[:, 0], hip_pos_arr[0, 0])
        by = np.append(hip_pos_arr[:, 1], hip_pos_arr[0, 1])
        bz = np.append(hip_pos_arr[:, 2], hip_pos_arr[0, 2])

        body_line.set_data(bx, by)
        body_line.set_3d_properties(bz)

        # 머리
        front_center = (hip_pos_arr[0] + hip_pos_arr[1]) / 2
        head_point.set_data([front_center[0]], [front_center[1]])
        head_point.set_3d_properties([front_center[2]])

        # 다리 (동적 발 위치 사용!)
        for key in keys:
            points = get_leg_points_dynamic(model, key, T_wb, current_feet[key])
            leg_lines[key].set_data(points[:, 0], points[:, 1])
            leg_lines[key].set_3d_properties(points[:, 2])

        return body_line, head_point

    anim = animation.FuncAnimation(fig, update, frames=frames, interval=50, blit=False)
    plt.close()
    return HTML(anim.to_jshtml())

# 실행
display(create_marching_animation(frames=60))

In [ ]:
# @title 4. 사인파 보행 (Sine Wave Trot)

def create_walking_animation(frames=80):
    model = MaicatModel()

    fig = plt.figure(figsize=(10, 6))
    ax = fig.add_subplot(111, projection='3d')

    # 바닥 그리드
    xx, yy = np.meshgrid(np.linspace(-0.3, 0.3, 10), np.linspace(-0.2, 0.2, 10))
    ax.plot_wireframe(xx, yy, np.zeros_like(xx), color='gray', alpha=0.1)

    ax.set_xlim(-0.25, 0.25) # 앞뒤 공간 확보
    ax.set_ylim(-0.2, 0.2)
    ax.set_zlim(0, 0.3)
    ax.set_xlabel('X (Front)')
    ax.set_ylabel('Y (Left)')
    ax.set_zlabel('Z (Up)')
    ax.view_init(elev=15, azim=-100) # 걷는 모습이 잘 보이도록 측면 뷰
    try: ax.set_box_aspect([1, 1, 1])
    except: pass

    # 그래픽 객체 초기화
    body_line, = ax.plot([], [], [], 'k-', linewidth=3, label='Body')
    head_point, = ax.plot([], [], [], 'go', markersize=10, label='Head')

    leg_lines = {}
    keys = ["FL", "FR", "RR", "RL"]
    for key in keys:
        line, = ax.plot([], [], [], color='blue', linewidth=4, marker='o', markersize=5, markeredgecolor='k')
        leg_lines[key] = line

    title = ax.set_title("Maicat Walking (Sine Wave)")

    # ---------------------------------------------------------
    # [핵심] 보행 궤적 생성 함수 (Gait Generator)
    # ---------------------------------------------------------
    def get_foot_trajectory(t, phase_offset):
        """
        t: 시간
        phase_offset: 다리 위상 (0 또는 PI)
        return: dx, dz (초기 발 위치 기준 변화량)
        """
        # 주기: 2*PI
        # 보폭(Stride): 8cm, 발 들기(Height): 3cm
        stride = 0.08
        step_h = 0.03

        # 현재 사이클 시간 (0 ~ 2PI)
        cycle_t = (t + phase_offset) % (2 * np.pi)

        dx = 0.0
        dz = 0.0

        # --- 1. Swing Phase (발을 들어 앞으로 보냄) ---
        # 0 ~ PI 구간 (전반부 50%)
        if cycle_t < np.pi:
            # 진행률 (0.0 ~ 1.0)
            progress = cycle_t / np.pi

            # X축: 뒤(-stride/2) -> 앞(+stride/2) (Cosine 활용)
            dx = -np.cos(progress * np.pi) * (stride / 2)

            # Z축: 바닥(0) -> 꼭대기(step_h) -> 바닥(0) (Sine 활용)
            dz = step_h * np.sin(progress * np.pi)

        # --- 2. Stance Phase (발을 딛고 뒤로 밈) ---
        # PI ~ 2PI 구간 (후반부 50%)
        else:
            # 진행률 (0.0 ~ 1.0)
            progress = (cycle_t - np.pi) / np.pi

            # X축: 앞(+stride/2) -> 뒤(-stride/2) (Linear Interpolation)
            # 몸이 앞으로 가는 효과를 냄
            dx = (stride / 2) - (stride * progress)

            # Z축: 바닥에 붙어있음
            dz = 0.0

        return dx, dz

    def update(frame):
        # 시간 t (속도 조절)
        t = frame * 0.25

        # --- [Gait Logic] ---
        # 트로트: 대각선 다리끼리 같은 위상
        # Group A (Phase 0): FL(앞왼), RR(뒤오)
        # Group B (Phase PI): FR(앞오), RL(뒤왼)

        current_feet = {}

        # 각 다리의 목표 위치 계산
        for key in keys:
            # 위상 결정
            phase = 0 if key in ["FL", "RR"] else np.pi

            # 궤적 계산
            dx, dz = get_foot_trajectory(t, phase)

            # 초기 발 위치에 변화량 적용
            pos = model._world_foot[key].copy()
            pos[0] += dx # X축(앞뒤) 이동
            pos[2] += dz # Z축(상하) 이동
            current_feet[key] = pos

        # --- [Body Motion] ---
        # 트로트 리듬에 맞춰 몸통 흔들기 (자연스러움)
        z_bob = 0.085 + 0.003 * np.sin(2 * t) # 위아래 (2배 빠른 주파수)
        p_rock = np.radians(2 * np.sin(t))    # 앞뒤 끄덕임 (Pitch)

        orn = [0, p_rock, 0]
        pos = [0, 0, z_bob]

        R_wb = RPY(orn[0], orn[1], orn[2])
        T_wb = RpToTrans(R_wb, pos)

        # --- [Visualization] ---
        # 몸통 업데이트
        hip_world_pos = []
        for key in keys:
            p_bh = model._body_to_hip[key]
            p_wh_homo = np.dot(T_wb, np.append(p_bh, 1))
            hip_world_pos.append(p_wh_homo[:3])
        hip_pos_arr = np.array(hip_world_pos)

        bx = np.append(hip_pos_arr[:, 0], hip_pos_arr[0, 0])
        by = np.append(hip_pos_arr[:, 1], hip_pos_arr[0, 1])
        bz = np.append(hip_pos_arr[:, 2], hip_pos_arr[0, 2])

        body_line.set_data(bx, by)
        body_line.set_3d_properties(bz)

        # 머리 업데이트
        front_center = (hip_pos_arr[0] + hip_pos_arr[1]) / 2
        head_point.set_data([front_center[0]], [front_center[1]])
        head_point.set_3d_properties([front_center[2]])

        # 다리 업데이트 (동적 위치)
        for key in keys:
            points = get_leg_points_dynamic(model, key, T_wb, current_feet[key])
            leg_lines[key].set_data(points[:, 0], points[:, 1])
            leg_lines[key].set_3d_properties(points[:, 2])

        return body_line, head_point

    # 프레임 수와 간격 조절로 자연스러운 속도 구현
    anim = animation.FuncAnimation(fig, update, frames=frames, interval=40, blit=False)
    plt.close()
    return HTML(anim.to_jshtml())

# 실행
display(create_walking_animation(frames=80))

In [ ]:
# @title 5. 베지어 곡선 보행 (Bezier Curve Trot)

def create_bezier_walking_animation(frames=80):
    model = MaicatModel()

    fig = plt.figure(figsize=(10, 6))
    ax = fig.add_subplot(111, projection='3d')

    # 바닥 및 뷰 설정
    xx, yy = np.meshgrid(np.linspace(-0.3, 0.3, 10), np.linspace(-0.2, 0.2, 10))
    ax.plot_wireframe(xx, yy, np.zeros_like(xx), color='gray', alpha=0.1)

    ax.set_xlim(-0.25, 0.25)
    ax.set_ylim(-0.2, 0.2)
    ax.set_zlim(0, 0.3)
    ax.set_xlabel('X (Front)')
    ax.set_ylabel('Y (Left)')
    ax.set_zlabel('Z (Up)')
    ax.view_init(elev=15, azim=-100) # 측면 뷰
    try: ax.set_box_aspect([1, 1, 1])
    except: pass

    # 그래픽 객체
    body_line, = ax.plot([], [], [], 'k-', linewidth=3, label='Body')
    head_point, = ax.plot([], [], [], 'go', markersize=10, label='Head')

    leg_lines = {}
    keys = ["FL", "FR", "RR", "RL"]
    for key in keys:
        line, = ax.plot([], [], [], color='blue', linewidth=4, marker='o', markersize=5, markeredgecolor='k')
        leg_lines[key] = line

    title = ax.set_title("Maicat Walking (Cubic Bezier Curve)")

    # ---------------------------------------------------------
    # [핵심] 3차 베지어 곡선 함수
    # ---------------------------------------------------------
    def cubic_bezier(t, P0, P1, P2, P3):
        """
        t: 0.0 ~ 1.0 사이의 진행률
        P0~P3: 제어점 (x, z)
        return: 계산된 위치 (x, z)
        """
        # 베지어 공식: B(t) = (1-t)^3*P0 + 3(1-t)^2*t*P1 + 3(1-t)t^2*P2 + t^3*P3
        term0 = (1-t)**3 * P0
        term1 = 3 * (1-t)**2 * t * P1
        term2 = 3 * (1-t) * t**2 * P2
        term3 = t**3 * P3
        return term0 + term1 + term2 + term3

    def get_foot_trajectory_bezier(t, phase_offset):
        # 보폭 및 높이 설정
        stride = 0.08  # 8cm
        lift_h = 0.04  # 4cm

        cycle_t = (t + phase_offset) % (2 * np.pi)

        dx = 0.0
        dz = 0.0

        # --- 1. Swing Phase (베지어 곡선 적용) ---
        if cycle_t < np.pi:
            # 진행률 (0.0 ~ 1.0)
            progress = cycle_t / np.pi

            # 제어점 정의 (X, Z) - 발의 초기위치(0,0) 기준 상대좌표
            # P0: 시작점 (뒤)
            P0 = np.array([-stride/2, 0.0])
            # P1: 수직 상승 (뒤 + 위) -> 직사각형 느낌을 위해 X는 유지하고 Z만 올림
            P1 = np.array([-stride/2, lift_h])
            # P2: 전진 및 하강 준비 (앞 + 위)
            P2 = np.array([stride/2, lift_h])
            # P3: 착지 (앞)
            P3 = np.array([stride/2, 0.0])

            # 베지어 계산
            pos = cubic_bezier(progress, P0, P1, P2, P3)
            dx = pos[0]
            dz = pos[1]

        # --- 2. Stance Phase (직선 후퇴) ---
        else:
            progress = (cycle_t - np.pi) / np.pi
            # X축: 앞(+stride/2) -> 뒤(-stride/2)
            dx = (stride / 2) - (stride * progress)
            # Z축: 바닥
            dz = 0.0

        return dx, dz

    def update(frame):
        t = frame * 0.25

        current_feet = {}
        for key in keys:
            phase = 0 if key in ["FL", "RR"] else np.pi

            # 베지어 궤적 함수 사용
            dx, dz = get_foot_trajectory_bezier(t, phase)

            pos = model._world_foot[key].copy()
            pos[0] += dx
            pos[2] += dz
            current_feet[key] = pos

        # Body Motion
        z_bob = 0.085 + 0.003 * np.sin(2 * t)
        p_rock = np.radians(2 * np.sin(t))

        orn = [0, p_rock, 0]
        pos = [0, 0, z_bob]

        R_wb = RPY(orn[0], orn[1], orn[2])
        T_wb = RpToTrans(R_wb, pos)

        # Visualization
        hip_world_pos = []
        for key in keys:
            p_bh = model._body_to_hip[key]
            p_wh_homo = np.dot(T_wb, np.append(p_bh, 1))
            hip_world_pos.append(p_wh_homo[:3])
        hip_pos_arr = np.array(hip_world_pos)

        bx = np.append(hip_pos_arr[:, 0], hip_pos_arr[0, 0])
        by = np.append(hip_pos_arr[:, 1], hip_pos_arr[0, 1])
        bz = np.append(hip_pos_arr[:, 2], hip_pos_arr[0, 2])

        body_line.set_data(bx, by)
        body_line.set_3d_properties(bz)

        front_center = (hip_pos_arr[0] + hip_pos_arr[1]) / 2
        head_point.set_data([front_center[0]], [front_center[1]])
        head_point.set_3d_properties([front_center[2]])

        for key in keys:
            points = get_leg_points_dynamic(model, key, T_wb, current_feet[key])
            leg_lines[key].set_data(points[:, 0], points[:, 1])
            leg_lines[key].set_3d_properties(points[:, 2])

        return body_line, head_point

    anim = animation.FuncAnimation(fig, update, frames=frames, interval=40, blit=False)
    plt.close()
    return HTML(anim.to_jshtml())

# 실행
display(create_bezier_walking_animation(frames=80))

In [ ]:
# @title 6. 'ㄷ'자 보행 (Boxy Gait) 시뮬레이션

def create_boxy_walking_animation(frames=80):
    model = MaicatModel()

    fig = plt.figure(figsize=(10, 6))
    ax = fig.add_subplot(111, projection='3d')

    xx, yy = np.meshgrid(np.linspace(-0.3, 0.3, 10), np.linspace(-0.2, 0.2, 10))
    ax.plot_wireframe(xx, yy, np.zeros_like(xx), color='gray', alpha=0.1)

    ax.set_xlim(-0.25, 0.25)
    ax.set_ylim(-0.2, 0.2)
    ax.set_zlim(0, 0.3)
    ax.set_xlabel('X (Front)')
    ax.set_ylabel('Y (Left)')
    ax.set_zlabel('Z (Up)')
    ax.view_init(elev=10, azim=-100) # 발 들림이 잘 보이는 각도
    try: ax.set_box_aspect([1, 1, 1])
    except: pass

    body_line, = ax.plot([], [], [], 'k-', linewidth=3, label='Body')
    head_point, = ax.plot([], [], [], 'go', markersize=10, label='Head')

    leg_lines = {}
    keys = ["FL", "FR", "RR", "RL"]
    for key in keys:
        line, = ax.plot([], [], [], color='blue', linewidth=4, marker='o', markersize=5, markeredgecolor='k')
        leg_lines[key] = line

    title = ax.set_title("Maicat 'Boxy' Walking (Exaggerated Bezier)")

    # 3차 베지어 함수
    def cubic_bezier(t, P0, P1, P2, P3):
        term0 = (1-t)**3 * P0
        term1 = 3 * (1-t)**2 * t * P1
        term2 = 3 * (1-t) * t**2 * P2
        term3 = t**3 * P3
        return term0 + term1 + term2 + term3

    def get_foot_trajectory_boxy(t, phase_offset):
        stride = 0.08
        lift_h = 0.05 # 발 높이를 좀 더 높임 (잘 보이게)

        cycle_t = (t + phase_offset) % (2 * np.pi)
        dx, dz = 0.0, 0.0

        # --- Swing Phase ---
        if cycle_t < np.pi:
            progress = cycle_t / np.pi

            # [과장된 제어점 설정]
            # X축 범위를 1.2배로 벌리고, 높이를 1.5배로 높여서
            # 그래프의 '어깨' 부분을 넓게 만듦 -> 평평한 구간이 길어짐
            P0 = np.array([-stride/2, 0.0])
            P1 = np.array([-stride/2 * 1.5, lift_h * 1.5]) # 과장됨
            P2 = np.array([stride/2 * 1.5, lift_h * 1.5])  # 과장됨
            P3 = np.array([stride/2, 0.0])

            pos = cubic_bezier(progress, P0, P1, P2, P3)
            dx = pos[0]
            dz = pos[1]

        # --- Stance Phase ---
        else:
            progress = (cycle_t - np.pi) / np.pi
            dx = (stride / 2) - (stride * progress)
            dz = 0.0

        return dx, dz

    def update(frame):
        t = frame * 0.25

        current_feet = {}
        for key in keys:
            phase = 0 if key in ["FL", "RR"] else np.pi
            dx, dz = get_foot_trajectory_boxy(t, phase)

            pos = model._world_foot[key].copy()
            pos[0] += dx
            pos[2] += dz
            current_feet[key] = pos

        # Body Motion
        z_bob = 0.085 + 0.003 * np.sin(2 * t)
        p_rock = np.radians(2 * np.sin(t))

        orn = [0, p_rock, 0]
        pos = [0, 0, z_bob]

        R_wb = RPY(orn[0], orn[1], orn[2])
        T_wb = RpToTrans(R_wb, pos)

        # Visualization
        hip_world_pos = []
        for key in keys:
            p_bh = model._body_to_hip[key]
            p_wh_homo = np.dot(T_wb, np.append(p_bh, 1))
            hip_world_pos.append(p_wh_homo[:3])
        hip_pos_arr = np.array(hip_world_pos)

        bx = np.append(hip_pos_arr[:, 0], hip_pos_arr[0, 0])
        by = np.append(hip_pos_arr[:, 1], hip_pos_arr[0, 1])
        bz = np.append(hip_pos_arr[:, 2], hip_pos_arr[0, 2])

        body_line.set_data(bx, by)
        body_line.set_3d_properties(bz)

        front_center = (hip_pos_arr[0] + hip_pos_arr[1]) / 2
        head_point.set_data([front_center[0]], [front_center[1]])
        head_point.set_3d_properties([front_center[2]])

        for key in keys:
            points = get_leg_points_dynamic(model, key, T_wb, current_feet[key])
            leg_lines[key].set_data(points[:, 0], points[:, 1])
            leg_lines[key].set_3d_properties(points[:, 2])

        return body_line, head_point

    anim = animation.FuncAnimation(fig, update, frames=frames, interval=40, blit=False)
    plt.close()
    return HTML(anim.to_jshtml())

# 실행
display(create_boxy_walking_animation(frames=80))

In [ ]:
# @title 7. 사인파 vs 베지어 궤적 비교 (2D 그래프)

import numpy as np
import matplotlib.pyplot as plt

def plot_trajectory_comparison():
    # 설정: 보폭 10cm, 높이 4cm
    stride = 0.10
    height = 0.04

    # 0~1 사이의 진행률 (t)
    t = np.linspace(0, 1, 100)

    # --- 1. 사인파 (Sine Wave) ---
    # X: -0.05 ~ 0.05 (Cosine)
    # Z: 0 ~ 0.04 ~ 0 (Sine)
    x_sine = -np.cos(t * np.pi) * (stride / 2)
    z_sine = height * np.sin(t * np.pi)

    # --- 2. 베지어 곡선 (Bezier) ---
    # 제어점 (Control Points)
    P0 = np.array([-stride/2, 0])       # 시작
    P1 = np.array([-stride/2, height])  # 수직 상승 유도
    P2 = np.array([stride/2, height])   # 수평 이동 유도
    P3 = np.array([stride/2, 0])        # 수직 하강 유도

    # 베지어 공식
    curve = []
    for val in t:
        point = (1-val)**3 * P0 + 3*(1-val)**2 * val * P1 + 3*(1-val) * val**2 * P2 + val**3 * P3
        curve.append(point)
    curve = np.array(curve)
    x_bezier = curve[:, 0]
    z_bezier = curve[:, 1]

    # --- 3. 과장된 베지어 (Exaggerated Box) ---
    # 제어점을 더 높고 넓게 벌려서 'ㄷ'자에 가깝게 만듦
    # P1, P2의 높이를 목표 높이보다 더 높게(1.3배) 설정하여 위쪽을 평평하게 만듦
    P1_ex = np.array([-stride/2 * 1.2, height * 1.3])
    P2_ex = np.array([stride/2 * 1.2, height * 1.3])

    curve_ex = []
    for val in t:
        point = (1-val)**3 * P0 + 3*(1-val)**2 * val * P1_ex + 3*(1-val) * val**2 * P2_ex + val**3 * P3
        curve_ex.append(point)
    curve_ex = np.array(curve_ex)

    # --- 그래프 그리기 ---
    plt.figure(figsize=(10, 5))

    plt.plot(x_sine, z_sine, label='Sine Wave (Semi-Circle)', color='blue', linestyle='--')
    plt.plot(x_bezier, z_bezier, label='Standard Bezier', color='green', linewidth=2)
    plt.plot(curve_ex[:,0], curve_ex[:,1], label='Exaggerated Bezier (Boxy)', color='red', linewidth=3)

    plt.title("Foot Tip Trajectory Comparison")
    plt.xlabel("X (Forward/Backward) [m]")
    plt.ylabel("Z (Height) [m]")
    plt.legend()
    plt.grid(True)
    plt.axis('equal') # 비율 고정
    plt.show()

plot_trajectory_comparison()